In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
from ulam import *

with UlamModel(stan_file="ulam_models/example.stan") as u:
    # Should go to the data block
    N = u.data('N', u.integer(lower=0))
    x = u.data('x', u.vector(N), missing_data=True)
    y = u.data('y', u.vector(N), missing_data=True)

    # Should go to the parameters block
    intercept = u.parameter('intercept', u.real())
    slope = u.parameter('slope', u.real())
    error = u.parameter('error', u.real(lower=0.01))

    # The block location for this for loop should be inferred automatically
    # from the analysis of the generated AST and form "type analysis" of
    # the variables
    for i in u.range('i', 1, N):
        # Uppercase because in Ulam distributions are uppercase and functions are lowercase
        y[i] ^ u.Normal(x[i] * slope + intercept, error)

14:16:45 - cmdstanpy - INFO - compiling stan file /home/tmbb/Projects/sideprojects/python/ulam/ulam_models/example.stan to exe file /home/tmbb/Projects/sideprojects/python/ulam/ulam_models/example
14:16:53 - cmdstanpy - INFO - compiled model executable: /home/tmbb/Projects/sideprojects/python/ulam/ulam_models/example


In [40]:
from cmdstanpy import CmdStanModel
import numpy
import pandas
import scipy.stats

slope = 0.89
intercept = -0.5
error = 0.3

N = 70
x_full = scipy.stats.norm.rvs(size=N, loc=2.5, scale=0.7)
y_full = scipy.stats.norm.rvs(size=N, loc=(x_full * slope + intercept), scale=error)

x = pandas.Series([(i % 5 == 2 and i % 2 == 1) and numpy.NaN or x_i for (i, x_i) in enumerate(x_full)])
y = pandas.Series([(i % 4 == 2 and i % 5 == 1) and numpy.NaN or y_i for (i, y_i) in enumerate(y_full)])

# y = scipy.stats.norm.rvs(size=N, loc=(x * slope + intercept), scale=error)
data = dict(N=N, x=x, y=y)

idata = u.sample(data=data)
idata

14:23:04 - cmdstanpy - INFO - CmdStan start processing


{'N': 70,
 'N__x__missing': 7,
 'N__x__not_missing': 63,
 'N__y__missing': 4,
 'N__y__not_missing': 66,
 'x__is_missing': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0]),
 'x__missing_data_index': [0,
                           1,
                           2,
                           3,
                           4,
                           5,
                           6,
                           0,
                           7,
                           8,
                           9,
                           10,
                           11,
                           12,
                           13,
                           14,
                           15,
                           1,
                           16,
                           17,
                           18,

chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

14:23:04 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
14:23:04 - cmdstanpy - ERROR - Chain [2] error: error during processing Operation not permitted
14:23:04 - cmdstanpy - ERROR - Chain [3] error: error during processing Operation not permitted
14:23:04 - cmdstanpy - ERROR - Chain [4] error: error during processing Operation not permitted


14:23:04 - cmdstanpy - INFO - CmdStan done processing.


RuntimeError: Error during sampling:
Exception: vector[uni] indexing: accessing element out of range. index 0 out of range; expecting index to be between 1 and 63 (in '/home/tmbb/Projects/sideprojects/python/ulam/ulam_models/example.stan', line 67, column 8 to column 127)
	Exception: vector[uni] indexing: accessing element out of range. index 0 out of range; expecting index to be between 1 and 63 (in '/home/tmbb/Projects/sideprojects/python/ulam/ulam_models/example.stan', line 67, column 8 to column 127)
Exception: vector[uni] indexing: accessing element out of range. index 0 out of range; expecting index to be between 1 and 63 (in '/home/tmbb/Projects/sideprojects/python/ulam/ulam_models/example.stan', line 67, column 8 to column 127)
	Exception: vector[uni] indexing: accessing element out of range. index 0 out of range; expecting index to be between 1 and 63 (in '/home/tmbb/Projects/sideprojects/python/ulam/ulam_models/example.stan', line 67, column 8 to column 127)
Exception: vector

In [ ]:
from matplotlib import pyplot

axes = arviz.plot_trace(
    idata,
    kind='rank_vlines',
    var_names=['intercept', 'slope', 'error']
)

fig = axes[0, 0].get_figure()
fig.tight_layout()

In [ ]:
u.plot_ppc(idata)

In [ ]:
import arviz

# idata = arviz.from_cmdstanpy(
#     posterior=fit,
#     posterior_predictive='y_hat',
#     observed_data={'y': y}
# )

# arviz.plot_ppc(idata, data_pairs={'y': 'y_hat'}, kind='kde')
arviz.plot_loo_pit(idata=idata, y='y', y_hat='y_hat', ecdf=True)

In [ ]:
import numpy
import pandas

def ith_element(i):
    if i % 4 == 1 or i % 7 == 2:
        return numpy.NaN
    else:
        return (i * i * 1.3 + 4.5) / (i - 0.8) 

s = pandas.Series([ith_element(i) for i in range(1, 30)])
s

In [ ]:
import os
import re
from typing import List
from dataclasses import dataclass

import jinja2

SURVIVAL_ANALYSIS_LIST = [
    'normal',
    'skew_normal',
    'cauchy',
    'student_t',
    'logistic',
    'gumbel',
    'inv_chi_square',
    'scaled_inv_chi_square',
    'inv_gamma',
    'lognormal',
    'weibull',
    'frechet',
    'rayleigh',
    'wiener',
    'pareto',
    'pareto_type_2',
    'exponential',
    'exp_mod_normal',
    'double_exponential',
    'gamma',
    'digamma'
]

@dataclass
class StanFunction:
    py_name: str
    stan_name: str
    arguments: List[str]
    argument_types: List[str]
    return_type: str

def full_paths_in_dir(directory):
    return [os.path.join(directory, path) for path in os.listdir(directory)]


def rename_builtins(name):
    if name == 'lambda':
        return 'lambda_'
    else:
        return name


def extract_functions_from_page_text(text):
    function_tuples = re.findall(r'<!--\s*([^;]+)\s*;\s*([^;]+)\s*;\s*\(([^)]+)\);\s*-->', text)
    functions = []

    for function_tuple in function_tuples:
        (return_type, fun, args_text) = function_tuple
        
        typed_args = [a.strip().rsplit(' ', 1) for a in re.split(r"[,\|]", args_text)]

        if ['...'] not in typed_args and ['array['] not in typed_args and fun != 'map_rect':
            arg_types = [typed_arg[0] for typed_arg in typed_args]
            args = [typed_arg[1] for typed_arg in typed_args]
            args = [rename_builtins(arg) for arg in args]

            function = StanFunction(
                py_name=fun,
                stan_name=fun,
                arguments=args,
                argument_types=arg_types,
                return_type=return_type
            )

            functions.append(function)

    return functions
    


def extract_functions_from_page(path):
    with open(path) as f:
        contents = f.read()
    
    return extract_functions_from_page_text(contents)

pages = [path for path in full_paths_in_dir('stan-docs/functions-reference') if path.endswith('.Rmd')]

template = jinja2.Template("""
from py_expr import PyExpr, python_to_ast
from stan_ast import FunctionCall

class StanFunctionsLibrary:
    def _function(self, function_name, args):
        arguments_as_ast = [python_to_ast(arg) for arg in args]
        function_call_ast = FunctionCall(function_name, arguments_as_ast)
        self.functions[function_name] = self.functions.get(function_name, 0) + 1

        return PyExpr(self, function_call_ast)
              
{% for f in functions %}
    def {{ f.py_name }}({% for arg in ['self'] + f.arguments %}{{ arg }}{{ ", " if not loop.last else "" }}{% endfor %}):
        return self._function('{{ f.stan_name }}', [{% for arg in f.arguments %}{{ arg }}{{ ", " if not loop.last else "" }}{% endfor %}])          
{% endfor %}
""")

def to_camel_case(snake_str):
    return "".join(x.capitalize() for x in snake_str.lower().split("_"))

def maybe_add_sampling_distribution_for_function(functions, function):
    if function.stan_name.endswith('_lpdf') or function.stan_name.endswith('_lpmf'):
        if function.stan_name.endswith('_lpdf'):
            suffix = '_lpdf'
        elif function.stan_name.endswith('_lpmf'):
            suffix = '_lpmf'

        base_name = function.stan_name[:-5]
        dist_name = to_camel_case(base_name)

        f_sampling = StanFunction(
            py_name=dist_name,
            stan_name=base_name,
            arguments=function.arguments[1:],
            argument_types=function.argument_types[1:],
            return_type=None
        )

        # Right censoring (for survival analysis)

        f_right_censored = StanFunction(
            py_name=dist_name + "_with_right_censoring",
            stan_name=base_name + "_with_right_censoring",
            arguments=function.arguments[1:],
            argument_types=function.argument_types[1:],
            return_type=None
        )

        f_right_censored_lpdf = StanFunction(
            py_name=base_name + "_with_right_censoring" + suffix,
            stan_name=base_name + "_with_right_censoring" + suffix,
            arguments=function.arguments + ['event'],
            argument_types=function.argument_types + ['reals'],
            return_type='reals'
        )
        
        f_right_censored_lupdf = StanFunction(
            py_name=base_name + "_with_right_censoring" + suffix,
            stan_name=base_name + "_with_right_censoring" + suffix,
            arguments=function.arguments + ['event'],
            argument_types=function.argument_types + ['reals'],
            return_type='reals'
        )

        f_right_censored_rng = StanFunction(
            stan_name=base_name + "_with_right_censoring_rng",
            py_name=base_name + "_with_right_censoring_rng",
            arguments=function.arguments + ['event'],
            argument_types=function.argument_types[1:] + ['reals'],
            return_type='reals'
        )

        # Left censoring (for survival analysis)
        
        f_left_censored = StanFunction(
            py_name=dist_name + "_with_left_censoring",
            stan_name=base_name + "_with_left_censoring",
            arguments=function.arguments[1:] + ['event'],
            argument_types=function.argument_types[1:] + ['reals'],
            return_type=None
        )

        f_left_censored_lpdf = StanFunction(
            py_name=base_name + "_with_left_censoring" + suffix,
            stan_name=base_name + "_with_left_censoring" + suffix,
            arguments=function.arguments + ['event'],
            argument_types=function.argument_types + ['reals', 'reals'],
            return_type='reals'
        )
        
        f_left_censored_lupdf = StanFunction(
            py_name=base_name + "_with_left_censoring" + suffix,
            stan_name=base_name + "_with_left_censoring" + suffix,
            arguments=function.arguments + ['event'],
            argument_types=function.argument_types + ['reals', 'reals'],
            return_type='reals'
        )

        f_left_censored_rng = StanFunction(
            py_name=base_name + "_with_left_censoring_rng",
            stan_name=base_name + "_with_left_censoring_rng",
            arguments=function.arguments + ['event'],
            argument_types=function.argument_types[1:] + ['reals'],
            return_type='reals'
        )

        # Right and left censoring (for survival analysis)
        
        f_left_and_right_censored = StanFunction(
            py_name=dist_name + "_with_left_and_right_censoring",
            stan_name=base_name + "_with_left_and_right_censoring",
            arguments=function.arguments[1:] + ['event_left', 'event_right'],
            argument_types=function.argument_types[1:] + ['reals', 'reals'],
            return_type=None
        )

        f_left_and_right_censored_lpdf = StanFunction(
            py_name=base_name + "_with_left_and_right_censoring" + suffix,
            stan_name=base_name + "_with_left_and_right_censoring" + suffix,
            arguments=function.arguments + ['event_left', 'event_right'],
            argument_types=function.argument_types + ['reals', 'reals'],
            return_type='reals'
        )
        
        f_left_and_right_censored_lupdf = StanFunction(
            py_name=base_name + "_with_left_and_right_censoring" + suffix,
            stan_name=base_name + "_with_left_and_right_censoring" + suffix,
            arguments=function.arguments + ['event_left', 'event_right'],
            argument_types=function.argument_types + ['reals', 'reals'],
            return_type='reals'
        )

        f_left_and_right_censored_rng = StanFunction(
            py_name=base_name + "_with_left_and_right_censoring_rng",
            stan_name=base_name + "_with_left_and_right_censoring_rng",
            arguments=function.arguments[1:] + ['event_left', 'event_right'],
            argument_types=function.argument_types[1:] + ['reals', 'reals'],
            return_type='reals'
        )


        if function.stan_name in SURVIVAL_ANALYSIS_LIST:
            new_functions = [
                function, f_sampling,
                f_left_censored, f_left_censored_lpdf, f_left_censored_lupdf, f_left_censored_rng,
                f_right_censored, f_right_censored_lpdf, f_right_censored_lupdf, f_right_censored_rng,
                f_left_and_right_censored, f_left_and_right_censored_lpdf, f_left_and_right_censored_lupdf,
                f_left_and_right_censored_rng
            ]
        
        else:
            new_functions = [function, f_sampling]

        functions.extend(new_functions)

    else:
        functions.append(function)

all_functions = []
for page in pages:
    functions = extract_functions_from_page(page)
    all_functions.extend(functions)

all_functions = [function for function in all_functions if not function.stan_name.startswith('operator')]

functions = []
for function in all_functions:
    maybe_add_sampling_distribution_for_function(functions, function)

function_dict = dict()
for function in functions:
    if function.stan_name in function_dict:
        pass
    else:
        function_dict[function.stan_name] = function

functions = sorted(function_dict.values(), key=lambda f: f.stan_name.lower())

open('stan_functions_library.py', 'w').write(template.render(functions=functions))